# Experiment tracking 📈

We can now try to improve our metrics in a structured manner.

But it's important to keep track of our progress and that of our colleagues, rathing than doing it in an ad-hoc manner and eye-balling the outputs. This is where experiment tracking comes in.

In this notebook, we'll track our experiments in the cloud, including inputs, prompts, outputs and metrics.

In [ ]:
# !uv add azureml-core
import azureml.core

In [5]:
from azureml.core import Workspace, Experiment
import os
import dotenv

dotenv.load_dotenv()  # This load our .env file as environment variables

SUBSCRIPTION_ID = os.environ["SUBSCRIPTION_ID"]
RESOURCE_GROUP = os.environ["RESOURCE_GROUP"]
WORKSPACE_NAME = os.environ["WORKSPACE_NAME"]
USER_NAME = os.environ["USER_NAME"]

In [ ]:
%

In [6]:
# Connect to Azure ML Workspace
ws = Workspace(subscription_id=SUBSCRIPTION_ID,
               resource_group=RESOURCE_GROUP,
               workspace_name=WORKSPACE_NAME)

In [7]:
# For autoreloading external modules
%load_ext autoreload
%autoreload 2

In [17]:
import time

from llmops_training.news_reader.data import get_evaluation_data
from llmops_training.news_reader.extraction import (
    extract_general_info_from_articles, 
    get_general_info_prompt_template,
)
from llmops_training.news_reader.evaluation import (
    evaluate_extract_general_info_success_rate, 
    evaluate_summarization, 
    evaluate_title,
)

In [18]:
eval_data = get_evaluation_data()

Let's start by creating an Experiment in Azure ML. Under this experiment we will create and track multiple runs.

In [9]:
experiment = Experiment(workspace=ws, name=f"llmops-experiment-{USER_NAME}")

We can start a new run:

In [19]:
run_name = USER_NAME + "-" + str(int(time.time()))
run = experiment.start_logging(name=run_name)

Run a part of the pipeline we want to evaluate:

In [20]:
general_info_prompt_template = get_general_info_prompt_template()
general_info_list = extract_general_info_from_articles(
    general_info_prompt_template, eval_data["article"].to_list()
)

Compute the metrics with the generated outputs of the previous step:

In [21]:
metrics = {
    "general_info_success_rate": evaluate_extract_general_info_success_rate(general_info_list),
    "title_accuracy": evaluate_title(general_info_list, eval_data),
    "summarization_rouge_1": evaluate_summarization(general_info_list, eval_data)["rouge-1"]
}
metrics

{'general_info_success_rate': 1.0,
 'title_accuracy': 1.0,
 'summarization_rouge_1': 0.3047572529066488}

Log some metrics and parameters:

In [22]:
run.log("general_info_prompt_template", general_info_prompt_template)
for m in metrics:
    run.log(m, metrics[m])

And stop the run:

In [23]:
run.complete()

> **Exercise** 📝
>
> - Run through the above code if you haven't already.
> - Explore the results in the Azure ML studio the Jobs section.
> - Improve the prompt and try to get better results.
> - Compare your results in the Azure ML Experiments section.
> - (Bonus) Extend the `evaluation.py` script to log metrics and parameters to Azure ML Experiments.